# Test 3 (+ test 5) - Blind vs non-blind, plotted over x-think

In [22]:
! tar czf Testing.tar *

In [ ]:
! nvidia-smi

In [ ]:
import textworld
import textworld.gym

In [ ]:
import time
import re
import os
from glob import glob
from typing import Mapping, Any

import pickle
import random

In [ ]:
import torch

torch.set_default_device('cuda')
torch.cuda.device("cuda")
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision('high')

In [11]:
!pip install imptools

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for imptools: filename=imptools-1.3.0-py3-none-any.whl size=5422 sha256=885c1cded71b8c3811109516c10ea35a98a92e8c8e1a186d8bfcb7456cda8ea8
  Stored in directory: c:\users\greg\appdata\local\pip\cache\wheels\a7\1b\f4\82873dac15e1260329d662bd50ea1bd0f693b573b805461d37
Successfully built imptools



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import sys
sys.path.append("../Self_evaluation")

from self_evaluation import play, LLMAgentSelfEvaluate

ModuleNotFoundError: No module named 'textworld'

## Game generation

In [ ]:
# 11 to 20 included
seeds = range(11,21)
max_scores = []

In [ ]:
!tw-make tw-simple --rewards dense --goal detailed --seed 11 --test --silent -f --output games/test-seed11.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 12 --test --silent -f --output games/test-seed12.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 13 --test --silent -f --output games/test-seed13.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 14 --test --silent -f --output games/test-seed14.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 15 --test --silent -f --output games/test-seed15.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 16 --test --silent -f --output games/test-seed16.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 17 --test --silent -f --output games/test-seed17.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 18 --test --silent -f --output games/test-seed18.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 19 --test --silent -f --output games/test-seed19.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 20 --test --silent -f --output games/test-seed20.z8

In [ ]:
# memes
seeds = [6, 10, 11, 20, 45, 46, 89, 1863, 79010123, 1179382318]
max_scores = [10, 7, 7, 7, 10, 7, 10, 7, 10, 8]

In [ ]:
!tw-make tw-simple --rewards dense --goal detailed --seed 6 --test --silent -f --output games/test-seed6.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 10 --test --silent -f --output games/test-seed10.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 11 --test --silent -f --output games/test-seed11.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 20 --test --silent -f --output games/test-seed20.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 45 --test --silent -f --output games/test-seed45.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 46 --test --silent -f --output games/test-seed46.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 89 --test --silent -f --output games/test-seed89.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 1863 --test --silent -f --output games/test-seed1863.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 79010123 --test --silent -f --output games/test-seed79010123.z8
!tw-make tw-simple --rewards dense --goal detailed --seed 1179382318 --test --silent -f --output games/test-seed1179382318.z8

## Game running

In [ ]:
max_steps = 100
n_episodes = 1
n_think_indices = range(0,11)

In [ ]:
for n in n_think_indices:
    for read_bool in (True, False):
        for seed in seeds:
            results = play(LLMAgentSelfEvaluate(selfeval_turns=n, reads_own_reasoning=read_bool, verbose=True),
                           f"games/test-seed{seed}", max_steps =max_steps,  n_episodes=n_episodes)
            with open(f'./Testing 3/{n}think_{"blind_" if not read_bool else ""}_seed{seed}.pickle', 'wb') as f:
                pickle.dump(results, f)
                print("Data pickled.")
                f.close()

# Data aggregation for visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bootstrap

In [ ]:
avg_final_scores = np.array()
avg_final_scores_ci = np.array()
avg_final_scores_blind = np.array()
avg_final_scores_blind_ci = np.array()

for n in n_think_indices:
    for read_bool in (True, False):
        avg_final_score = 0
        final_scores = []
        for seed in seeds:
            results = []
            with open(f'./Testing 3/{n}think_{"blind_" if not read_bool else ""}_seed{seed}.pickle', 'rb') as f:
                pickle.load(results, f)
                print("Data pickled.")
                f.close()
            final_scores.append(results[0][-1][1] / max_scores[seeds.index(seed)]) # run 1, last step, score normalized

        avg_final_score = np.mean(final_scores)
        bootstrap_results = bootstrap(data=(final_scores,), 
                              statistic=np.mean,
                              method="basic",
                              n_resamples=1000,
                              confidence_level=0.9)
        if read_bool:
            avg_final_scores.append(avg_final_score)
            avg_final_scores_ci.append((bootstrap_results.confidence_interval.high, bootstrap_results.confidence_interval.low)) 
        else:
            avg_final_scores_blind.append(avg_final_score)
            avg_final_scores_blind_ci.append((bootstrap_results.confidence_interval.high, bootstrap_results.confidence_interval.low)) 

# Visualization

In [9]:
figsize=(7,7)

In [ ]:
avg_final_scores = np.array()
avg_final_scores_ci = np.array()
avg_final_scores_blind = np.array()
avg_final_scores_blind_ci = np.array()

In [ ]:
x = n_think_indices # n-think

fig, ax = plt.subplots(figsize=figsize)

ax.plot(n, avg_final_scores, label="non-ephemeral self-evaluation", marker=".", linestyle="-", color="red")
ax.plot(n, avg_final_scores_blind, label="ephemeral self-evaluation", marker=".", linestyle="-", color="blue")

ax.fill_between(n,
                np.clip([i[0] for i in avg_final_scores_ci], 0, 1),
                np.clip([i[1] for i in avg_final_scores_ci], 0, 1),
                alpha=0.1, color="red")
ax.fill_between(n,
                np.clip([i[0] for i in avg_final_scores_blind_ci], 0, 1),
                np.clip([i[1] for i in avg_final_scores_blind_ci], 0, 1),
                alpha=0.1, color="blue")


ax.set_title("Average final score of an ephemeral/non-ephemeral $n$-think model with $n=0,...,10$")
ax.set_xlabel('n (number non-self-evaluating turns for every self-evaluating turn)')
ax.set_ylabel('average final score, normalized')

gridlines = np.arange(n[0], n[-1]+1, 1)
ax.set_xticks(gridlines)
ax.grid(axis="x", alpha=0.2)

# ax.set_ylim(0.2, 1.1)

ax.legend()
plt.show()